In [413]:
# Initialize notebook
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from matplotlib import pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)

METADATA = {'Contributor': 'T. Dunn'}
FSIZE = [5,4]
FDPI = 100
SAVEFIGS = True

df0 = pd.read_csv('monster-immunities.csv')
df0 = df0.astype({'Book': 'category', 'Category': 'category', 'Legendary': 'category', 'Type': 'category', 'Size': 'category'})
df0['All Conditions'] = df0['Blinded'] + df0['Charmed'] + df0['Deafened'] + df0['Exhaustion'] + df0['Frightened'] + df0['Grappled'] + df0['Incapacitated'] + df0['Invisible'] + df0['Paralyzed'] + df0['Petrified'] + df0['Poisoned'] + df0['Prone'] + df0['Restrained'] + df0['Stunned'] + df0['Unconscious']
df0.drop_duplicates(inplace=True)
print('Includes {} monsters in database'.format(len(df0.index.to_list())))

CONDITIONS = ['Blinded','Charmed','Deafened','Exhaustion','Frightened',
    'Grappled','Incapacitated','Invisible','Paralyzed','Petrified',
    'Poisoned','Prone','Restrained','Stunned','Unconscious']
CONDITION_COLORS = ['gold', 'red', 'green', 'orange', 'teal',
    'purple', 'navy', 'maroon', 'olivedrab', 'orangered',
    'blue', 'darkviolet', 'dodgerblue', 'brown', 'darkgreen']

Includes 1419 monsters in database


In [134]:
# Setup template and html config for plotly figures.
import plotly.graph_objects as go

TFB_TEMPLATE = dict(
    layout=go.Layout(
        template='plotly_white',
        autosize=True, # must be True to auto-scale when resizing
        margin=dict(l=50, r=25, b=40, t=20, pad=4),
        font=dict(
            family='sans-serif',
            size=14
        ),
        hovermode='closest',
        xaxis=dict(
            autorange=False,
            automargin=True,
            fixedrange=True,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            tickmode='linear',
            ticks='outside',
            minor=dict(
                #showgrid=True,
                tickmode='linear',
                ticks='outside',
            ),
            zeroline=False,
        ),
        yaxis=dict(
            autorange=False,
            automargin=True,
            fixedrange=True,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            tickmode='linear',
            ticks='outside',
            minor=dict(
                #showgrid=True,
                tickmode='linear',
                ticks='outside',
            ),
            zeroline=False,
        ),
        legend=dict(
            #bordercolor='Black',
            #borderwidth=1,
            font_size=10,
            #tracegroupgap=5,
            #orientation='h',
        )
    )
)

TFB_CONFIG = {
    'responsive': True, # must be True to auto-scale when resizing
    'autosizable': True, # doesn't impact auto rescaling
    'showAxisDragHandles': False,
    'displaylogo': False,
    'displayModeBar': 'hover',
    'modeBarButtonsToRemove': [
        'select2d',
        'lasso2d',
        'zoom2d',
        'zoomIn2d',
        'zoomOut2d',
        'pan2d',
        'autoScale2d',
        'hoverClosestCartesian',
        'hoverCompareCartesian',
        'toggleSpikelines',
        'resetScale2d',
    ],
    'toImageButtonOptions': {
        'format': 'png', # one of png, svg, jpeg, webp
        'filename': 'tfb-plot',
        'height': 450,
        'width': 600,
        'scale': 2
    },
}


In [383]:
# Plots the average probability that a monster is immune to each condition
df1 = df0[df0['Category'].isin(['generic','legendary','unique']) & df0['CR'].between(1/8, 30)]

conditions = CONDITIONS.copy()

# create custom text
total_monsters = df1.shape[0]
total_immune = [int(df1[c].sum().tolist()) for c in conditions]
pct_immune = [100*df1[c].mean().tolist() for c in conditions]
custom_text = [f'{p:.0f}% ({i} out of {total_monsters})' for p, i in zip(pct_immune, total_immune)]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=conditions,
    y=pct_immune,
    mode='markers', 
    marker=dict(size=12, line=dict(width=1, color='Black')),
    text=custom_text,
    unselected=dict(marker=dict(opacity=0.0)),
    hovertemplate=
        '<b>Condition:</b> %{x}<br>'+
        '<b>Immune:</b> %{text}'+
        '<extra></extra>'
))

fig.update_layout(
    template=TFB_TEMPLATE,
    #width=500,
    #height=400,
    xaxis=dict(
        #title_text='condition',
        range=[-0.5, len(conditions)-0.5],
        #tick0=0, dtick=1,
        #minor=dict(tick0=0, dtick=1),
    ),
    yaxis=dict(
        title_text='percent immune',
        range=[-1, 30],
        tick0=0, dtick=5,
        minor=dict(tick0=0, dtick=2.5),
    )
)

fig.show(config=TFB_CONFIG)
if SAVEFIGS:
    fig.write_html(
        f'./fig-condition-immunity-probability.html', 
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )

In [346]:
# Calculates the probability of a monster for each being immune to each condition.
df1 = df0[df0['Category'].isin(['generic','legendary','unique']) & df0['CR'].between(1/8, 30)]

# get the total number of monsters from each type
dfG = df1.groupby(['Type']).count().reset_index()
type_totals = dfG['Book'].to_list()
print('Total number of monsters:', np.sum(type_totals))

# get the number of monsters immune to each condition for each type
dfG = df1.groupby(['Type']).sum().reset_index()

# construct custom text for each cell
custom_texts = []
for c in CONDITIONS:
    type_immune = dfG[c].to_list()
    custom_texts.append([f'{i:.0f} out of {t}' for i, t in zip(type_immune, type_totals)])

# get percent of each type immune to each condition
dfG = df1.groupby(['Type']).mean().reset_index()
z = []
conditions = CONDITIONS.copy()
for condition in conditions:
    row = dfG[condition].tolist()
    row = [int(100*v) for v in row]
    z.append(row)

x = [x.lower() for x in dfG['Type'].tolist()]
y = [x.lower() for x in conditions]

fig = go.Figure()
fig.add_trace(go.Heatmap(
    z=z, x=x, y=y,
    zmin=0, zmax=120,
    hoverongaps=False,
    colorscale='blues',
    text=custom_texts,
    texttemplate='%{z:.0f}',
    hovertemplate='<b>Type:</b> %{x}<br>'+
                  '<b>Condition:</b> %{y}<br>'+
                  '<b>Immune:</b> %{z:.0f}% (%{text})'+
                  '<extra></extra>'
))

fig.update_traces(showscale=False)

fig.update_layout(
    template=TFB_TEMPLATE,
    font=dict(
        family='sans-serif',
        size=12
    ),
    xaxis=dict(
        title_text='',
        autorange=True,
        showgrid=False,
    ),
    yaxis=dict(
        title_text='',
        autorange=True,
        showgrid=False,
    ),
)

fig.show(config=TFB_CONFIG)
if SAVEFIGS:
    file_name = f'./fig-condition-immunity-probability-heat-map.html'
    fig.write_html(
        file_name, 
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    # add style to figure
    with open(file_name, 'r') as fin:
        fig_soup = BeautifulSoup(fin, 'html.parser')

    fig_soup.div['style'] = 'width:1000px; min-width:50%; max-width:100%; max-height:500px'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

Total number of monsters: 1182


In [411]:
# Plots the percent of monsters immune to each condition as a function of their CR.
df1 = df0[df0['Category'].isin(['generic','legendary','unique']) & df0['CR'].between(1/8, 30)]
dfG = df1.groupby('CR').mean().reset_index()

conditions = CONDITIONS.copy()
colors = CONDITION_COLORS.copy()
i = conditions.index('Invisible')
conditions.pop(i); colors.pop(i)
i = conditions.index('Incapacitated')
conditions.pop(i); colors.pop(i)

fig = go.Figure()

for condition, color in zip(conditions, colors):
    # generate custom text
    cr_totals = df1.groupby('CR').count()['Book'].to_list()
    condition_totals = [int(x) for x in df1.groupby('CR').sum()[condition].to_list()]
    custom_text = [f'({i} out of {t})' for i, t in zip(condition_totals, cr_totals)]

    # plot data
    fig.add_trace(go.Scatter(
        x=dfG['CR'],
        y=100*dfG[condition],
        mode='markers', 
        line=dict(color=color, dash='solid'),
        line_shape='spline',
        unselected=dict(marker=dict(opacity=0.0)),
        showlegend=True,
        name=condition,
        legendgroup=condition,
        text=custom_text,
        hovertemplate=
            '<b>CR:</b> %{x}<br>'+
            '<b>Condition:</b> ' + condition + '<br>'+
            '<b>Immune:</b> %{y:.0f}% %{text}'+
            '<extra></extra>'
    ))

    # add fit line
    coefs = np.polyfit(dfG['CR'], 100*dfG[condition], 1)
    poly = np.poly1d(coefs)
    fig.add_trace(go.Scatter(
        x=dfG['CR'],
        y=poly(dfG['CR']),
        mode='lines', 
        line=dict(color=color, dash='solid'),
        unselected=dict(marker=dict(opacity=0.0)),
        showlegend=False,
        legendgroup=condition,
        legendgrouptitle_text='fit',
        hoverinfo='skip',
    ))

fig.update_layout(
    template=TFB_TEMPLATE,
    xaxis=dict(
        title_text='challenge rating',
        range=[0.0, 30.2],
        tick0=0, dtick=5,
        minor=dict(tick0=0, dtick=1),
    ),
    yaxis=dict(
        title_text='percent immune',
        range=[-5, 115],
        tick0=0, dtick=20,
        minor=dict(tick0=0, dtick=10),
    ),
    legend=dict(
        yanchor='top',  y=0.99,
        xanchor='left', x=0.01,
        orientation='v',
        tracegroupgap=0,
    )
)

fig.show(config=TFB_CONFIG)
if SAVEFIGS:
    file_name = f'./fig-condition-immunity-prob-vs-cr.html'
    fig.write_html(
        file_name, 
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    # add style to figure
    with open(file_name, 'r') as fin:
        fig_soup = BeautifulSoup(fin, 'html.parser')

    fig_soup.div['style'] = 'width:700px; min-width:50%; max-width:100%'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [412]:
# Plots average number of conditional immunities for monsters at each CR.
df1 = df0[df0['Category'].isin(['generic','legendary','unique']) & df0['CR'].between(1/8, 30)]
dfG = df1.groupby('CR').mean()

conditions = ['All Conditions']
colors = ['black']

fig = go.Figure()

for condition, color in zip(conditions, colors):
    dfR = dfG[condition].rolling(1, center=True).mean()
    fig.add_trace(go.Scatter(
        x=dfR.index.to_series(),
        y=dfR,
        mode='lines+markers', 
        line=dict(color=color, dash='solid'),
        unselected=dict(marker=dict(opacity=0.0)),
        name=condition,
        hovertemplate=
            'CR %{x}<br>'+
            'immunities %{y:.2f}'+
            '<extra></extra>'
    ))

fig.update_layout(
    template=TFB_TEMPLATE,
    xaxis=dict(
        title_text='challenge rating',
        range=[0.0, 30.2],
        tick0=0, dtick=5,
        minor=dict(tick0=0, dtick=1),
    ),
    yaxis=dict(
        title_text='condition immunities',
        range=[-0.05, 6],
        tick0=0, dtick=1.0,
        minor=dict(tick0=0, dtick=0.25),
    ),
    legend=dict(
        yanchor='top',  y=0.99,
        xanchor='left', x=0.01,
        orientation='v',
    )
)

fig.show(config=TFB_CONFIG)

In [239]:
# Plots the average number of condition immunities for monsters of each type.
condition = 'All Conditions'

df1 = df0[df0['Category'].isin(['generic','legendary','unique']) & df0['CR'].between(1, 30)]
dfG = df1.groupby(['Type']).mean().reset_index()

fig = go.Figure()

fig.add_trace(go.Histogram(
    histfunc='avg', 
    x=dfG['Type'], 
    y=dfG[condition], 
    name=condition,
    hovertemplate=
        '<b>%{x}:</b> %{y:.2f}<br>'+
        '<extra></extra>',
))

fig.update_layout(
    template=TFB_TEMPLATE,
    xaxis=dict(
        title_text='',
        range=[-0.5, 14-0.5],
    ),
    yaxis=dict(
        title_text=f'condition immunities',
        range=[0.0, 7.0],
        tick0=0, dtick=1.0,
        minor=dict(tick0=0, dtick=0.5),
    )
)

fig.show(config=TFB_CONFIG)

In [124]:
# Runs a linear regression fit for immunity to one condition based on monster CR and type.
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

yCol = 'Poisoned'
xCols = ['CR','Type']
aCols = [yCol] + xCols
df1 = df0[aCols]
X = pd.get_dummies(data=df1[xCols], drop_first=False)
Y = df1[yCol]

model = LinearRegression()
model.fit(X, Y)

coeff_parameter = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])

# generate report
X_Sm = sm.add_constant(X)
ls = sm.OLS(Y, X_Sm).fit()
print(ls.summary())

                            OLS Regression Results                            
Dep. Variable:               Poisoned   R-squared:                       0.576
Model:                            OLS   Adj. R-squared:                  0.572
Method:                 Least Squares   F-statistic:                     142.1
Date:                Sat, 17 Dec 2022   Prob (F-statistic):          1.42e-260
Time:                        23:25:13   Log-Likelihood:                -305.16
No. Observations:                1478   AIC:                             640.3
Df Residuals:                    1463   BIC:                             719.8
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.2296      0.014  